# Stage 2 Usage Examples (LangChain Reasoning & Router)

This notebook demonstrates how to use the Stage 2 chains to query the synthetic payments dataset and obtain concise summaries.

- Routed Q&A via `router_chain.ask()`
- Direct code generation and execution via `query_chain`
- Summarization via `summary_chain`
- Optional: enabling LLM with `OPENAI_API_KEY` (otherwise heuristic fallback is used)


In [1]:
# optional: load .env for local development
#%pip install -q python-dotenv
from dotenv import load_dotenv
load_dotenv()  # loads variables from .env into the process env


True

In [2]:
import logging
logging.basicConfig(level=logging.INFO, format='[%(name)s] %(levelname)s: %(message)s')

In [3]:
# Setup imports and paths
import os, sys
from pathlib import Path
import pandas as pd

# Ensure project root on path
PROJECT_ROOT = str(Path.cwd().parent)
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print('Project root:', PROJECT_ROOT)
print('OPENAI_API_KEY set:', bool(os.getenv('OPENAI_API_KEY')))

DATA_PATH = str(Path(PROJECT_ROOT) / 'data' / 'payments.csv')
print('Data path:', DATA_PATH)


Project root: /home/andres/Documents/chat-payments
OPENAI_API_KEY set: True
Data path: /home/andres/Documents/chat-payments/data/payments.csv


In [4]:
print("LLM_PROVIDER =", os.getenv("LLM_PROVIDER"))
print("LLM_MODEL    =", os.getenv("LLM_MODEL"))
print("GROQ_API_KEY set?", bool(os.getenv("GROQ_API_KEY")))

LLM_PROVIDER = groq
LLM_MODEL    = None
GROQ_API_KEY set? True


In [5]:
# Load dataset preview
df = pd.read_csv(DATA_PATH, parse_dates=['timestamp'])
print(f"Rows: {len(df):,} | Columns: {len(df.columns)}")
df.head(3)


Rows: 10,000 | Columns: 11


,transaction_id,user_id,segment,country,merchant,category,amount,timestamp,device_type,is_refunded,is_fraudulent
0,879463009885,208,SMB,AR,Microsoft,electronics,126.02,2023-01-01 01:04:39+00:00,tablet,False,False
1,404737561214,183,consumer,US,Telcel,telco,48.31,2023-01-01 03:08:19+00:00,tablet,False,False
2,12503321040,155,consumer,BR,Oxxo,retail,10.56,2023-01-01 08:52:04+00:00,mobile,False,False


In [6]:
# Example 1: Routed data Q&A
from src.chains.router_chain import ask

q1 = "Which merchants had the highest total revenue last month?"
resp1 = ask(q1)
print('Route:', resp1['route'])
print('Answer:', resp1['answer'])

# Convert table (list of dicts) back into a DataFrame for display
if resp1.get('table'):
    pd.DataFrame(resp1['table']).head(10)


[src.chains.llm_factory] INFO: *** PROVIDER NAME: groq
[src.chains.llm_factory] INFO: *** MODEL NAME: llama3-70b-8192
/home/andres/Documents/chat-payments/src/chains/query_chain.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.llm_chain = LLMChain(llm=llm, prompt=sql_prompt_template)
[src.chains.query_chain] INFO: QueryChain initialized with LLM provider=groq model=auto
[src.chains.query_chain] INFO: QueryChain using LLM for question: Which merchants had the highest total revenue last month?
/home/andres/Documents/chat-payments/src/chains/query_chain.py:131: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self.llm_chain.run({"question": question})  # type: ignore
[httpx] INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HT

[router] llm_used=True provider=groq model=auto
Route: data
Answer: Summary for 'Which merchants had the highest total revenue last month?': 10 rows. Top rows:
| merchant   |   amount |
|:-----------|---------:|
| Apple      |  75567.6 |
| BestBuy    |  57285.5 |
| Microsoft  |  41248.8 |
| Delta      |  34953.3 |
| Airbnb     |  32691.9 |


In [13]:
# Example 2: Another routed question
q2 = "Which country has the highest average transaction amount?"
resp2 = ask(q2)
print('Route:', resp2['route'])
print('Answer:', resp2['answer'])

if resp2.get('table'):
    pd.DataFrame(resp2['table']).head(10)


[src.chains.query_chain] INFO: QueryChain initialized without LLM; using heuristic fallback
[src.chains.query_chain] INFO: QueryChain using heuristic fallback for question: Which country has the highest average transaction amount?
[src.chains.summary_chain] INFO: SummaryChain initialized without LLM; using heuristic fallback
[src.chains.summary_chain] INFO: SummaryChain using heuristic fallback for question: Which country has the highest average transaction amount?


[router] llm_used=False provider=- model=-
Route: data
Answer: Summary for 'Which country has the highest average transaction amount?': 5 rows. Top rows:
| country   |   amount |
|:----------|---------:|
| US        |  49.8392 |
| BR        |  46.391  |
| AR        |  41.9231 |
| CL        |  41.3484 |
| MX        |  38.164  |


In [14]:
# Example 3: Direct query chain usage
from src.chains.query_chain import run as run_query

q3 = "Top 10 merchants by total revenue"
res3 = run_query(q3)
print('Keys:', list(res3.keys()))

# Reconstruct DataFrame from 'split' orient
split = res3['table']
df3 = pd.DataFrame(split['data'], columns=split['columns'])
df3.head(10)


[src.chains.query_chain] INFO: QueryChain initialized without LLM; using heuristic fallback
[src.chains.query_chain] INFO: QueryChain using heuristic fallback for question: Top 10 merchants by total revenue


Keys: ['answer', 'table']


,transaction_id,user_id,segment,country,merchant,category,amount,timestamp,device_type,is_refunded,is_fraudulent
0,879463009885,208,SMB,AR,Microsoft,electronics,126.02,2023-01-01 01:04:39+00:00,tablet,False,False
1,404737561214,183,consumer,US,Telcel,telco,48.31,2023-01-01 03:08:19+00:00,tablet,False,False
2,12503321040,155,consumer,BR,Oxxo,retail,10.56,2023-01-01 08:52:04+00:00,mobile,False,False
3,176693838346,89,consumer,CL,Amazon,e-commerce,43.70,2023-01-01 11:25:07+00:00,desktop,False,False
4,276091832029,921,merchant,MX,Airbnb,travel,139.69,2023-01-01 12:43:28+00:00,mobile,False,False
5,719575841950,854,consumer,BR,Shell,fuel,57.99,2023-01-01 13:12:01+00:00,desktop,False,False
6,966074701384,422,consumer,US,Steam,gaming,23.45,2023-01-01 15:40:19+00:00,mobile,False,False
7,291922513770,373,consumer,BR,DoorDash,food_delivery,11.44,2023-01-01 15:54:56+00:00,mobile,False,False
8,813015498665,784,consumer,US,Netflix,subscriptions,13.86,2023-01-01 20:05:01+00:00,mobile,False,False
9,463845325312,669,consumer,US,Uber,mobility,69.49,2023-01-01 21:17:45+00:00,desktop,False,False


In [15]:
# Example 4: Summarize a result explicitly
from src.chains.summary_chain import SummaryChain

summary = SummaryChain().run(q3, df3)
print(summary)


[src.chains.summary_chain] INFO: SummaryChain initialized without LLM; using heuristic fallback
[src.chains.summary_chain] INFO: SummaryChain using heuristic fallback for question: Top 10 merchants by total revenue


Summary for 'Top 10 merchants by total revenue': 20 rows. Top rows:
|   transaction_id |   user_id | segment   | country   | merchant   | category    |   amount | timestamp                 | device_type   | is_refunded   | is_fraudulent   |
|-----------------:|----------:|:----------|:----------|:-----------|:------------|---------:|:--------------------------|:--------------|:--------------|:----------------|
|     879463009885 |       208 | SMB       | AR        | Microsoft  | electronics |   126.02 | 2023-01-01 01:04:39+00:00 | tablet        | False         | False           |
|     404737561214 |       183 | consumer  | US        | Telcel     | telco       |    48.31 | 2023-01-01 03:08:19+00:00 | tablet        | False         | False           |
|      12503321040 |       155 | consumer  | BR        | Oxxo       | retail      |    10.56 | 2023-01-01 08:52:04+00:00 | mobile        | False         | False           |
|     176693838346 |        89 | consumer  | CL        | Amazon    

In [16]:
# Example 5 (optional): Async ask()
# If the notebook environment has an event loop, fallback to sync.
try:
    import asyncio
    from src.chains.router_chain import ask_async
    
    def run_async_example():
        q = "What was the total payment volume last week?"
        try:
            resp = asyncio.run(ask_async(q))
        except RuntimeError:
            # Fallback if event loop is already running
            resp = ask(q)
        print('Route:', resp['route'])
        print('Answer:', resp['answer'])
        return resp
    
    _ = run_async_example()
except Exception as e:
    print('Async example skipped:', e)


[src.chains.query_chain] INFO: QueryChain initialized without LLM; using heuristic fallback
[src.chains.query_chain] INFO: QueryChain using heuristic fallback for question: What was the total payment volume last week?
[src.chains.summary_chain] INFO: SummaryChain initialized without LLM; using heuristic fallback
[src.chains.summary_chain] INFO: SummaryChain using heuristic fallback for question: What was the total payment volume last week?


[router] llm_used=False provider=- model=-
Route: data
Answer: Summary for 'What was the total payment volume last week?': 20 rows. Top rows:
|   transaction_id |   user_id | segment   | country   | merchant   | category    |   amount | timestamp                 | device_type   | is_refunded   | is_fraudulent   |
|-----------------:|----------:|:----------|:----------|:-----------|:------------|---------:|:--------------------------|:--------------|:--------------|:----------------|
|     879463009885 |       208 | SMB       | AR        | Microsoft  | electronics |   126.02 | 2023-01-01 01:04:39+00:00 | tablet        | False         | False           |
|     404737561214 |       183 | consumer  | US        | Telcel     | telco       |    48.31 | 2023-01-01 03:08:19+00:00 | tablet        | False         | False           |
|      12503321040 |       155 | consumer  | BR        | Oxxo       | retail      |    10.56 | 2023-01-01 08:52:04+00:00 | mobile        | False         | False      

/tmp/ipykernel_197656/4134056275.py:13: RuntimeWarning: coroutine 'ask_async' was never awaited
  resp = ask(q)
